# Исследование надежности заемщиков

## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [ ]:
import pandas as pd# импортируйте библиотеку pandas

In [ ]:
data=pd.read_csv('/datasets/data.csv')# прочитайте csv-файл
print(data.shape)

(21525, 12)


**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [ ]:
print(data.head(20))# ваш код здесь

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
for i in data. columns:
  print (i,data[i].isna().sum())# ваш код здесь

children 0
days_employed 2174
dob_years 0
education 0
education_id 0
family_status 0
family_status_id 0
gender 0
income_type 0
debt 0
total_income 2174
purpose 0


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [ ]:
print (data.isnull().sum()) # ваш код здесь

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [ ]:
income_type_inf=data.groupby('income_type').agg({'total_income':['median']})
ind=income_type_inf.index.tolist()
col=income_type_inf['total_income']['median'].tolist()
dict_empties=dict(zip(ind, col))
data['total_income']=data['total_income'].fillna(1)

for i in range(len(data['total_income'])):
  if data.loc[i,'total_income']==1:
    data.loc[i,'total_income']=dict_empties.get(data.loc[i,'income_type'])

#print (data.loc[:,['total_income','income_type']].head(35))# ваш код здесь

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [ ]:
data['days_employed']=data['days_employed'].abs()
#print (data.head(25))# ваш код здесь

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
print(data.groupby('income_type')['days_employed'].median())# ваш код здесь

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [ ]:
print (data['children'].unique())
#print(data['children'].value_counts())# ваш код здесь

[ 1  0  3  2 -1  4 20  5]


**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
d1=(data['children']!=20)&(data['children']!=-1)
data=data.loc[d1]
#print(data.shape)
#print (data.head(30))# ваш код здесь

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
print (data['children'].unique())# ваш код здесь


[1 0 3 2 4 5]


### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [ ]:
days_inf=data.groupby('income_type')['days_employed'].median()
data['days_employed']=data['days_employed'].fillna(1)
row=data.loc[data['days_employed']==1,'days_employed']
row2=data.loc[data['days_employed']==1,'income_type']
n=pd.concat([row, row2], axis=1)
k=[]
for i in n.loc[:,"income_type"]:
    k. append(days_inf[i])
n['days_employed']=k
data.loc[data['days_employed']==1,'days_employed']=n['days_employed']# ваш код здесь


**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [ ]:
print (data ['days_employed'].isnull(). sum())# ваш код здесь

0


### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income']=data['total_income'].astype('int') # ваш код здесь


### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.**

In [ ]:
data['education']= data['education'].str.lower()   # ваш код здесь
#print(data.shape)

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их. Сбрасывать индексы после удаления строк дубликатов с помощью `reset_index(drop=True)` здесь не требуется.**

In [ ]:
print (data.duplicated().sum())# посчитайте дубликаты

71


In [ ]:
data=data.drop_duplicates()# удалите дубликаты

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(row):
  if (row>=0 and row<=30000):
    return 'E'
  elif (row>=30001 and row<=50000):
    return 'D'
  elif (row>=50001 and row<=200000):
    return 'C'
  elif (row>=200001 and row<=1000000):
    return 'B'
  return 'A'   # создайте функцию categorize_income()

In [ ]:
data['total_income_category']=data['total_income'].apply(categorize_income)   # примените функцию методом apply()

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
print (data ['purpose'].unique())  # ваш код здесь

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [ ]:
def categorize_purpose(row):
  if 'автомоб' in row:
      return 'операции с автомобилем'
  elif 'недвиж' in row:
      return 'операции с недвижимостью'
  elif 'жиль' in row:
      return 'операции с недвижимостью'
  elif 'свадь' in row:
      return 'проведение свадьбы'
  elif 'образ' in row:
      return 'получение образования'# создайте функцию categorize_purpose()

In [ ]:
data['purpose_category']=data['purpose'].apply(categorize_purpose)
print(data.shape)# примените функцию методом apply()

(21331, 14)
